# Fetch Google travel times

In [1]:
import urllib2, json
import pandas as pd
import numpy as np
import datetime, time
from SECRET_KEY import secret_key

In [2]:
# Load TAZ records with x and y coordinates attached
taz = pd.read_csv(r'..\data\taz_xy.txt')

meter_mile_conversion = 0.000621371

- define extraction time (for future travel time predictions)
- set skim as auto

In [3]:
# Create a Google-formatted coordinates field
taz['g_coord'] = taz['y_gps'].astype('str') + ',' + taz['x_gps'].astype('str')



# set standard departure time for tomorrow at 8 AM
dep_hr = 8
dep_time = datetime.datetime.now()

# This only works if its in the middle of the month
# dep_time = dep_time.replace(hour=dep_hr,day=dep_time.day+1)
# dep_time = str(int(time.mktime(dep_time.timetuple())))

dep_time = dep_time.replace(hour=dep_hr,day=1,month=10)
dep_time = str(int(time.mktime(dep_time.timetuple())))

# Skims can be auto, transit, bike, or walk
mode = 'auto'

# Look up series of travel times for a single origin

In [22]:
otaz_list = [500,450]
dtaz_list = [550,600,650,750,850,950,1000]

In [108]:
results = {}
# Look up 
for otaz in otaz_list:
    print 'fetching times from zone: ' + str(otaz)
    
    urlfeed = ""

    origin = taz[taz['ID'] == otaz]['g_coord'].values[0]
    # get list of different destinations
    destination = ''

    for dtaz in dtaz_list:
        if otaz != dtaz:    # skip intrazonal trips where otaz==dtaz
            destination += taz[taz['ID'] == dtaz]['g_coord'].values[0] + '|'
#             dtaz_list.append(dtaz)
        # remove trailing |
    destination = destination[:-1]

    urlfeed += "https://maps.googleapis.com/maps/api/distancematrix/json?origins="+origin+"&destinations="+destination+ \
            "&mode="+mode+"&departure_time="+dep_time+"&key="+secret_key+"&units=imperial"

    # Fetch url and store
    results[otaz] = json.loads(urllib2.urlopen(urlfeed).read())

fetching times from zone: 500
fetching times from zone: 450


In [119]:
# Parse the results
results_dict = {}
results_dfs = {}
df = pd.DataFrame()
for otaz, data in results.iteritems():

    results_dict[otaz] = {}
    
    # loop through each destination
    i = 0
    for dtaz in dtaz_list:

        results_dict[otaz][dtaz] = {}
        results_dict[otaz][dtaz]['dtaz'] = dtaz_list[i]
        results_dict[otaz][dtaz]['dist'] = results[otaz]['rows'][0]['elements'][i]['distance']['value']*0.000621371    # convert meters to miles
        results_dict[otaz][dtaz]['time_ff'] = results[otaz]['rows'][0]['elements'][i]['duration']['value']/60.0    # convert seconds to minutes
        results_dict[otaz][dtaz]['time_cong'] = results[otaz]['rows'][0]['elements'][i]['duration_in_traffic']['value']/60.0    # congested
        
        i+=1
        
    tempdf = pd.DataFrame(results_dict[otaz]).T
    tempdf['otaz'] = otaz
    tempdf['dtaz'] = tempdf.index
    tempdf = tempdf.reset_index(drop=True)
    
    df = pd.concat([df,tempdf])

In [121]:
df

,dist,dtaz,time_cong,time_ff,otaz
0,1.724305,550,9.300000,10.483333,450
1,2.742732,600,13.883333,15.766667,450
2,2.915473,650,8.400000,9.183333,450
3,7.029570,750,13.033333,13.733333,450
4,10.974655,850,18.316667,19.333333,450
5,12.610103,950,16.983333,17.600000,450
6,13.514819,1000,20.766667,21.516667,450
0,0.937649,550,6.366667,7.233333,500
1,1.802597,600,10.200000,11.250000,500
2,2.372394,650,7.316667,8.133333,500
